In [ ]:
!pip3 install networkx

In [ ]:
import networkx as nx

In [ ]:
import pandas as pd

In [ ]:
import json 
from tqdm import tqdm

In [ ]:
fr = open("./contacts.json",'rb')
obj = json.load(fr)

In [ ]:
obj

In [ ]:
df = pd.DataFrame(obj)
df.head()

In [ ]:
(df.Phone == '').sum()

In [ ]:
email_dict = df.groupby('Email')['Id'].apply(list).to_dict()
phone_dict = df.groupby('Phone')['Id'].apply(list).to_dict()
orderId_dict = df.groupby('OrderId')['Id'].apply(list).to_dict()

In [ ]:
del email_dict['']
del phone_dict['']
del orderId_dict['']

In [ ]:
import networkx as nx
G = nx.Graph()

In [ ]:
G.add_nodes_from(df.Id.tolist())


In [ ]:
def genEdges(ll):
    r = []
    for i in range(len(ll)):
#         print(i,len(ll))
        for j in range(i+1,len(ll)):
            if i == j : break
#             print(ll[i],'->',ll[j],end=', ')
            r.append([ll[i],ll[j]])
#         print()
    return r
genEdges([1,2,3,4,5])

In [ ]:
email_edges = []
for e in email_dict:
    email_edges.extend(genEdges(email_dict[e]))
    
G.add_edges_from(email_edges)

In [ ]:
phone_edges = []
for e in phone_dict:
    phone_edges.extend(genEdges(phone_dict[e]))
    
G.add_edges_from(phone_edges)

In [ ]:
order_edges = []
for e in orderId_dict:
    order_edges.extend(genEdges(orderId_dict[e]))
    
G.add_edges_from(order_edges)

In [ ]:
components = list(nx.connected_components(G))


In [ ]:


temp_df = df.iloc[list(components[2]),:]

In [ ]:
temp_df

In [ ]:
def com2str(temp_df):
    Id_list = temp_df.Id.tolist()
    Id_list.sort()
#     Id_list
    total_conacts=temp_df.Contacts.sum()
#     total_conacts
    column2_str = "-".join([str(x) for x in Id_list])+", "+str(total_conacts)
    return column2_str

In [ ]:
com2str(temp_df)

In [ ]:
df['ticket_trace/contact']=['']*5000*100
assert (df['ticket_trace/contact']=='').sum() ==5000*100

In [ ]:

for i in tqdm(range(len(components))):
    id_list = list(components[i])
    temp_df = df.iloc[id_list,:]
    column2_str = com2str(temp_df)
    
    for ii in id_list:
        df.at[ii,'ticket_trace/contact']= column2_str
#     df.iloc[id_list,:]['ticket_trace/contact'] = [column2_str]*len(id_list)

In [ ]:
assert (df['ticket_trace/contact']=='').sum() ==0

In [ ]:
output_df = df[['Id','ticket_trace/contact']]
output_df.columns = ['ticket_id','ticket_trace/contact']
output_df

In [ ]:
output_df.to_csv("answer.csv",index=False,quotechar='\"')